In [1]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("sushant_report.pdf")

docs = loader.load()

In [3]:
doc = docs[0]

In [6]:
len(docs)

5

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

chunks = text_splitter.split_documents(docs)

In [8]:
len(docs), len(chunks)

(5, 11)

In [8]:
len(docs[0].page_content), len(chunks[0].page_content)

(618, 604)

In [9]:
! pip install faiss-cpu

In [10]:
from langchain_community.embeddings import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [11]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url="http://localhost:11434")

C:\Users\arraj\AppData\Local\Temp\ipykernel_19484\912281862.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url="http://localhost:11434")


In [12]:
single_vector = embeddings.embed_query("this is some text data")
index = faiss.IndexFlatL2(len(single_vector))

In [13]:
index = faiss.IndexFlatL2(len(single_vector))
index.ntotal, index.d

(0, 768)

In [14]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [15]:
ids = vector_store.add_documents(documents=chunks)

In [16]:
vector_store.index_to_docstore_id
len(ids)

11

In [17]:
question = "what is the system architecture"
docs = vector_store.search(query=question, search_type='similarity')

for doc in docs:
    print(doc.page_content)
    print("\n\n")

3.0 Hardware / Software Description 
Scalability in location-based applications depends on an efficient infrastructure, robust 
software platforms, and effective data management strategies. Here’s an in-depth look at the 
tools and setup for achieving scalability in LBAs. 
Software Platforms and Technologies: 
• 
Database Systems: 
o 
Distributed Databases: Tools like MongoDB, Cassandra, and Google Bigtable offer 
horizontal scalability, managing data across multiple nodes to handle large user 
loads. 
o 
Spatial Databases: PostgreSQL with PostGIS, or MongoDB’s geospatial capabilities, 
help with storing and querying location data, such as coordinates, distances, and 
map layers. 
• 
Cloud Services: AWS, Google Cloud, and Azure provide infrastructure for load balancing, 
distributed data storage, and real-time data processing, essential for scaling LBAs. 
• 
Caching Systems: Redis and Memcached are used for caching frequently accessed data,



especially with the advent of 5G, the adop

In [18]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs = {'k': 3, 
                                                                          'fetch_k': 100,
                                                                          'lambda_mult': 1})

In [19]:
docs = retriever.invoke(question)

In [20]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

In [21]:
model = ChatOllama(model="llama2", base_url="http://localhost:11434")

model.invoke("hi")

C:\Users\arraj\AppData\Local\Temp\ipykernel_19484\3530216374.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model="llama2", base_url="http://localhost:11434")


AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama2', 'created_at': '2025-01-22T11:20:51.9166733Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 7978886700, 'load_duration': 6965858400, 'prompt_eval_count': 21, 'prompt_eval_duration': 64313000, 'eval_count': 26, 'eval_duration': 930833000}, id='run-f96d57b5-8016-48b0-b0e4-5703d0849dd3-0')

In [22]:
prompt = hub.pull("rlm/rag-prompt")

c:\Users\arraj\miniconda3\envs\torch\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [24]:
prompt = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
    Always give in markdown format
    Question: {question} 
    Context: {context} 
    Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [25]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [26]:
rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [29]:
question = "what is are the evaluation metrics"

output = rag_chain.invoke(question)
print(output)

Here are the evaluation metrics mentioned in the context:

* Latency: The time taken to process and respond to user requests is crucial for LBAs. Low latency is achieved by using caching and optimized data structures for geospatial queries.
* Throughput: Measures the number of requests handled per second, a key indicator of an app’s ability to handle large user volumes.
* Scalability: Assessed based on how well the system can handle increased workloads, using techniques like sharding and load balancing.
* Fault Tolerance: Ensuring that the system remains operational even if certain servers or data centers fail.
* Input/Output:
	+ Input: User location data, including latitude, longitude, and additional metadata (speed, altitude).
	+ Output: Responses like maps, nearby places, route information, and personalized location-based recommendations.

The report also mentions the following strategies for improving scalability in location-based apps:

* Horizontal scaling: Adding more servers or